In [5]:
# Day 17 - Pyroclastic Flow, part 1
# It's bloody tetris.

# Input defines the movements left or right
# Standard sequence of block addition and initial placement

from operator import itemgetter




# Board will be the map of everything no longer moving. Include the floor.
# Each block will be (row, column) and floor will be row 0.



class Wind:
    '''
        Use this to track the state of the wind and keep state between function calls
    '''
    def __init__(self) -> None:
        self.wind = [x for x in open('input.txt').read().strip()]
        self.wind_pos = 0
        self.wind_len = len(self.wind)


    def GetWind(self):
        if self.wind[self.wind_pos] == '<':
            ret = -1
        else:
            ret = 1

        self.wind_pos = self.wind_pos + 1 if (self.wind_pos +1) < self.wind_len else 0
        return ret



def NewShape(pattern, max_height):
    '''
    Create a list with the positions of each block making up the new shape.
    Use an int to track which shape gets added next, 0-4:
        - + _| | #
    '''
    newshape = []

    if pattern == 0:
        # horizontal line
        newshape = [(max_height+4, x) for x in range(2, 6)]

    if pattern == 1:
        # plus sign
        newshape = [(max_height+4+1, x) for x in range(2, 5)]
        newshape.extend([(max_height+4+x, 3) for x in range(0,3,2)])

    if pattern == 2:
        # reverse L
        newshape = [(max_height+4, x) for x in range(2, 5)]
        newshape.extend([(max_height+4+x, 4) for x in range(1, 3)])


    elif pattern == 3:
        # vertical line
        newshape = [(max_height+4+x, 2) for x in range(4)]

    elif pattern == 4:
        # 2x2 square
        newshape = [(max_height+4+r, c+2) for r in range(2) for c in range(2)]

    return newshape



def BlowShape(shape, direction, board):
    '''
    Direction will be -1 if < and 1 if >
    '''

    temp = shape.copy()
    shape = [(r, c + direction) for r, c in shape]

    if (min(shape, key=itemgetter(1))[1] < 0) or (max(shape, key=itemgetter(1))[1] > 6) or any(coord in shape for coord in board):
        return temp
    else:
        return shape

    


def DownShape(shape):
    return [(r-1, c) for r, c in shape]



def PrintBoard(board):
    top = max(board, key=itemgetter(0))[0] + 2

    for r in range(top, -1, -1):
        print('|', end = '')
        for c in range(7):
            if (r, c) in board:
                print('#', end = '')
            else:
                print('.', end='')
        print('|')

    print()
    print()
        


def Main(blocks):
    b = []
    h = [0]

    board = set()
    board.update([(0, x) for x in range(7)])
    max_height = 0

    wind = Wind()


    for i in range(blocks):

        shape = NewShape(i%5, max_height)
        can_move = True

        while can_move:

            shape = BlowShape(shape, wind.GetWind(), board)
            temp = shape.copy()
            shape = DownShape(shape)

            if any(coord in shape for coord in board):
                can_move = False
                board.update(temp)

        max_height = max(board)[0]
        b.append(i)
        h.append(max_height)

        # PrintBoard(board)
    return b, h

In [6]:
import pandas as pd

In [39]:
# Create a dataframe with the heights for the first 5000 blocks

b, h = Main(5000)
h.pop()
df = pd.DataFrame({'blocks': b, 'height': h})


In [9]:
# Label the blocks 0 - 4 depending on shape
# Add the height added with this block

df['mod5_b'] = df['blocks'] % 5
df['diff_h'] = df['height'].diff()

In [33]:
# Export the data to csv

df.to_csv('real_data.csv', columns=['mod5_b','height', 'diff_h'])

In [32]:
# Create a column showing height added in the past 5 block cycle
df['diff_h5'] = df[df['mod5_b'] == 0][['height']].diff()

# Export this - this was good for finding the actual pattern
df[df['mod5_b'] == 0][['blocks', 'height', 'diff_h', 'diff_h5']].to_csv('mod5.csv')

At this point we have the csv exports of data for trying to find the pattern.

To use this with the example data then just change the filepath in the Main() function.

*Values Found*

Cycle starts at position 590

Cycle lasts 1760 blocks

Each cycle adds height 2737.

## Calculate Answer

In [42]:
start = int(df[:590]['diff_h'].sum())

repeats = int(1000000000000-589)//1760

remainder = (1000000000000-589) % 1760
rem_sum = df[590: (590+remainder)]['diff_h'].sum()


total = start + (repeats * 2737) + rem_sum
print(int(total))

1555113636385
